In [1]:
import os

In [2]:
%pwd

'c:\\Users\\sam\\End-to-End-Fashion-Recommendation-System-with-MLOps\\research'

In [3]:
import os
os.chdir('../')

In [41]:
%pwd

'c:\\Users\\sam\\End-to-End-Fashion-Recommendation-System-with-MLOps'

In [42]:
from pathlib import Path
from dataclasses import dataclass
import pickle
import regex as re
from nltk.corpus import stopwords
import pandas as pd

In [43]:
from src.recommendation_system.utils.common import read_yaml , create_dir
from src.recommendation_system.logging import logger
from src.recommendation_system.constants import CONFIG_PATH
from src.recommendation_system.logging import logger

In [44]:
@dataclass
class feature_enginerring:
    transformed_model: Path
    featured_data:   Path
    model_path:      Path


In [45]:
class Congif_manager:

    def __init__(self, config =CONFIG_PATH):
        self.config = read_yaml(config)

        create_dir([self.config.artifacts_root])

    def get_feature_engineering(self) -> feature_enginerring:

        config = self.config.feature_engineering          

        create_dir([config.featured_data,                 
                config.model_path])

        feature_engineering_config = feature_enginerring( 
        transformed_model = config.transformed_data,
        featured_data    = config.featured_data,
        model_path       = config.model_path
    )

        return feature_engineering_config   
        


In [ ]:
class feature_enginerring_config:

    def __init__(self, config ) -> feature_enginerring:
        self.config = config

    noise_words_v1 = {
        'men', 'mens',
        'shirt', 'tshirt',
        'regular', 'fit',
        'sleeve', 'neck',
        'solid', 'wear',
        'full', 'half',
        'stylish', 'man', 'round'
    }

    def clean_product_names(self, df: pd.DataFrame) -> pd.DataFrame:

        # lowercase
        df['product_name'] = df['product_name'].str.lower()

        # remove special chars
        df['product_name'] = df['product_name'].apply(
            lambda x: re.sub(r'[^a-zA-Z\s]', '', str(x))
        )

        # fix extra whitespace
        df['product_name'] = df['product_name'].apply(
            lambda x: re.sub(r'\s+', ' ', x).strip()
        )

        # remove english stopwords
        def stop_words(text):
            co = stopwords.words('english')
            return ' '.join(
                i for i in text.split() if i not in co
            )

        df['product_name'] = df['product_name'].apply(stop_words)

        # create style_text_v1
        def remove_noise(text):
            return ' '.join(
                w for w in text.split()
                if w not in self.noise_words_v1
            )

        df['style_text_v1'] = df['product_name'].apply(remove_noise)

        logger.info("[Step 1] Cleaning DONE")
        logger.info("product_name[0]  : %s", df['product_name'].iloc[0])
        logger.info("style_text_v1[0] : %s", df['style_text_v1'].iloc[0])

        return df

    def initiate_feature_engineering(self):
        logger.info("===== Feature Engineering STARTED =====")

        # load pkl
        df = pickle.load(
            open(self.config.transformed_model, 'rb')
        )
        logger.info("Loaded pkl | shape=%s", df.shape)

        # step 1 only
        df = self.clean_product_names(df)

        logger.info("Columns now : %s", list(df.columns))
        logger.info("Sample output :\n%s",
            df[['product_name', 'style_text_v1']].head(3).to_string()
        ) 

In [47]:
con = Congif_manager()
feature_eng = con.get_feature_engineering()
feature_eng = feature_enginerring_config(feature_eng)
feature_eng.initiate_feature_engineering()

[2026-02-27 22:47:20,527: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-02-27 22:47:20,530: INFO: common: created directory at: artifacts]
[2026-02-27 22:47:20,533: INFO: common: created directory at: artifacts/feature_engineering/featured_data/]
[2026-02-27 22:47:20,537: INFO: common: created directory at: artifacts/feature_engineering/model/]
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "box/box.py", line 605, in box.box.Box.__getitem__
KeyError: 'transformed_data'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "box/box.py", line 644, in box.box.Box.__getattr__
  File "box/box.py", line 632, in box.box.Box.__getitem__
box.exceptions.BoxKeyError: "'transformed_data'"

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "box/box.py", line 646, in box.box.Box.__getattr__
AttributeError: 'Box' object has no attribute 'transformed_data'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\sam\End-to-End-Fashion-Recommendation-System-with-MLOps\venv\Lib\site-packages\IPython\core\interactiveshell.py", line 3701, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\sam\AppData\Local\Temp\ipykernel_22892\224991083